In [1]:
import os
from dotenv import load_dotenv

# Load biến môi trường từ file .env (file .env cần nằm trong cùng thư mục với notebook)
load_dotenv()

# Lấy giá trị của TOGETHER_API_KEY từ biến môi trường
together_api_key = os.getenv('TOGETHER_API_KEY')
if not together_api_key:
    raise ValueError("Chưa tìm thấy TOGETHER_API_KEY trong file .env!")

elastic_api_key = os.getenv('ELASTIC_API_KEY')
if not elastic_api_key:
    raise ValueError("Chưa tìm thấy ELASTIC_API_KEY trong file .env!")

elastic_cloud_id = os.getenv('ELASTIC_CLOUD_ID')
if not elastic_cloud_id:
    raise ValueError("Chưa tìm thấy ELASTIC_CLOUD_ID trong file .env!")

tavily_api_key = os.getenv('TAVILY_API_KEY')
if not tavily_api_key:
    raise ValueError("Chưa tìm thấy TAVILY_API_KEY trong file .env!")


serper_api_key = os.getenv('SERPER_API_KEY')
if not serper_api_key:
    raise ValueError("Chưa tìm thấy SERPER_API_KEY trong file .env!")


print(" Key đã được thiết lập thành công!")


 Key đã được thiết lập thành công!


In [2]:
print(together_api_key)
print(elastic_api_key)
print(elastic_cloud_id)

ba4066853f6d39efbdb8e58274802acdb950955adb52d523b36cad88c5493aa1
LUVTcUlKWUIyR2JxbU84a1dRYVo6TFhidjQ4VlVRRmFYNUJQY2MweFVEUQ==
My_Elasticsearch_deployment:YXAtZWFzdC0xLmF3cy5lbGFzdGljLWNsb3VkLmNvbTo0NDMkNjAyNDBhMzU5NGU5NDIzNjkzMmJjYjQwMWYyZWZmMTMkNmQxNjEyNjAwZWI1NDQzZmE2ODA1YTI1YWQ5M2Y3ZTA=


In [ ]:
from langchain_together import ChatTogether, TogetherEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo
#
#meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo" )
result = llm.invoke("Thủ đô Việt Nam là thành phố nào? ")
print(result.content)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
from tqdm.notebook import tqdm
embd = HuggingFaceEmbeddings(model_name="VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")

In [ ]:
embd

## Load Elasticsearch Store



In [6]:
from langchain.vectorstores import ElasticsearchStore
index_name = "ecom-law0"

In [ ]:
# Khởi tạo connection tới Elasticsearch thông qua Elastic Cloud
vector_store = ElasticsearchStore(
    es_cloud_id=elastic_cloud_id,
    es_api_key=elastic_api_key,
    index_name=index_name,
    embedding=embd,
)

In [ ]:
indices = vector_store.client.cat.indices(format="json")
for index_info in indices:
    print(index_info["index"])


In [ ]:
num_docs = vector_store.client.count(index=index_name)['count']
print(f"Number of documents in the vectorstore: {num_docs}")


## Hybrid Retrieval

In [10]:
from typing import Dict
from langchain_elasticsearch import ElasticsearchRetriever
from elasticsearch import Elasticsearch

In [11]:
index_name = "ecom-law0"
text_field = "text"
es_url = "https://my-elasticsearch-deployment.es.ap-east-1.aws.elastic-cloud.com"

In [12]:
# --- Định nghĩa hàm hybrid_query ---
def hybrid_query(search_query: str) -> Dict:
    # Sử dụng embed_query để tạo vector cho truy vấn
    query_vector = embd.embed_query(search_query)
    return {
        "retriever": {
            "rrf": {  # Rank Reciprocal Fusion: kết hợp hai truy vấn theo kiểu "should"
                "retrievers": [
                    {  # Thành phần full text search: dùng match query trên trường "text"
                        "standard": {
                            "query": {
                                "match": {
                                    text_field: search_query
                                }
                            }
                        }
                    },
                    {  # Thành phần vector search: dùng knn query trên trường "embedding_field"
                        "knn": {
                            "field": "embedding_field",  # Điều chỉnh nếu mapping của bạn dùng tên khác
                            "query_vector": query_vector,
                            "k": 5,
                            "num_candidates": 10
                        }
                    }
                ]
            }
        }
    }


In [ ]:
# --- Thiết lập Elasticsearch client với xác thực ---
from elasticsearch import Elasticsearch
headers = {"Authorization": "ApiKey " + elastic_api_key.strip()}
es_client = Elasticsearch(
    hosts=[es_url],
    headers=headers
)
print("ElasticSearch Cluster Info:")
print(es_client.info())

In [14]:
from langchain_elasticsearch import ElasticsearchRetriever
hybrid_retriever = ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=hybrid_query,
    content_field=text_field,  # Trường chứa nội dung văn bản trong index
    url=es_url,
    api_key=elastic_api_key.strip()
)




## Prompting

In [15]:
from langchain.prompts import PromptTemplate

prompt_template = """
Bạn là một chuyên gia pháp lý hàng đầu, chuyên sâu về **Luật Thương mại của Việt Nam**, đồng thời am hiểu đầy đủ và chính xác các **bộ luật khác của Việt Nam** như: Bộ luật Dân sự, Luật Doanh nghiệp, Luật Trọng tài thương mại, Luật Đầu tư, Luật Sở hữu trí tuệ, v.v.

Bạn có nhiều năm kinh nghiệm giảng dạy, tư vấn và giải quyết tranh chấp pháp lý trong môi trường thực tiễn tại Việt Nam.

---

### ❓ Câu hỏi pháp lý:
{query}

---

### 📚 Thông tin pháp lý được truy xuất từ văn bản:
(Chỉ sử dụng thông tin trong phần này. Nếu không đủ căn cứ, hãy nói rõ.)
------------------------------------------------
{retrieved_context}
------------------------------------------------

---

### 🎯 Nhiệm vụ của bạn:

1. **Phân tích câu hỏi** để xác định vấn đề pháp lý cốt lõi và các quy định cần xem xét.
2. **Chỉ dựa trên thông tin được truy xuất ở trên**, đưa ra phân tích pháp lý chi tiết, có **trích dẫn đúng điều/khoản/bộ luật** theo định dạng chuẩn: `"Điều X, Khoản Y, Luật Z: ..."`.
3. **Tuyệt đối không suy diễn hoặc tự tạo nội dung luật** nếu không có trong context.
4. Nếu có nhiều cách hiểu/phương án, hãy liệt kê và **so sánh khách quan** (ưu/nhược điểm).
5. **Đưa ví dụ minh họa** cụ thể nếu có thể.
6. **Tổng hợp và trả lời rõ ràng, có logic**, trình bày theo **cấu trúc với tiêu đề phụ**.

---

### 📝 Trả lời:
"""


In [16]:
prompt = PromptTemplate(template=prompt_template, input_variables=["retrieved_context", "query"])

## Retrieval


### Test 1

In [17]:
# --- Thực hiện retrieval ---
query = "Theo luật trọng tài thương mại, trọng tài nước ngoài là gì?"
retrieved_docs = hybrid_retriever.invoke(query)

In [ ]:
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])
print(retrieved_context)

In [19]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)

In [ ]:
print(final_prompt)

In [ ]:
response = llm(final_prompt)
print(response.content)

### Test 2

In [22]:

# --- Thực hiện retrieval ---
query = "Theo luật trọng tài thương mại, Quyết định trọng tài?"
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [23]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)

In [ ]:
response = llm(final_prompt)
print(response.content)

### Test 3

In [26]:
# --- Thực hiện retrieval ---
query = '''Theo luật trọng tài thương mại,Đối với tranh chấp không có yếu tố nước ngoài, Hội đồng trọng tài có được áp dụng 
pháp luật Việt Nam để giải quyết tranh chấp hay không ?'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [27]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)

In [ ]:
response = llm(final_prompt)
print(response.content)

### Test 4

In [29]:
# --- Thực hiện retrieval ---
query = '''Luật trọng tài thương mại do ai ban hành'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [ ]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

### Test 5

In [31]:
# --- Thực hiện retrieval ---
query = '''Phán quyết của trọng tài trong luật trọng tài thương mại có hiệu lực như thế nào'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [ ]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

### Test 6

In [33]:
# --- Thực hiện retrieval ---
query = '''Sự khác biệt giữa trọng tài thương mại và toà án là gì'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [ ]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

### Test 7

In [35]:
# --- Thực hiện retrieval ---
query = '''Trong luật trọng tài thương mại, Phán quyết trọng tài có giá trị pháp lí như thế nào?'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [ ]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

### Test 8


In [59]:
# # --- Thực hiện retrieval ---
# query = "Theo luật trọng tài thương mại, Quyết định trọng tài?"
# retrieved_docs = hybrid_retriever.invoke(query)
# retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [58]:
# final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
# response = llm(final_prompt)
# print(response.content)

In [57]:
# print(final_prompt)

## Search Tools

### Tavily Searrh

In [22]:
from langchain_community.tools import TavilySearchResults
from pydantic import BaseModel, Field
from typing import List

In [23]:
# Định nghĩa model cho một mục kết quả
class SearchResultItem(BaseModel):
    url: str = Field(..., description="Link của bài viết")
    content: str = Field(..., description="Nội dung tóm tắt hoặc câu trả lời")

# Định nghĩa model cho output của tool
class TavilySearchOutput(BaseModel):
    results: List[SearchResultItem] = Field(..., description="Danh sách kết quả tìm kiếm")

In [ ]:
# Cấu hình tool với args_schema là model trên (nếu tool hỗ trợ tùy chỉnh schema)
tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    name="TavilyVietnamEcommerceLawSearch",
    description=(
        "Công cụ tìm kiếm chuyên dụng cho các vấn đề liên quan đến luật thương mại điện tử Việt Nam. "
        "Công cụ này trả về câu trả lời, tiêu đề, nội dung thô và link của các bài viết liên quan, "
        "hỗ trợ kết quả từ cả các nguồn tiếng Việt và tiếng Anh."
    ),
)

# Sử dụng truy vấn tiếng Việt
raw_result = tool.invoke({
    "query": "theo luật trọng tài thương mại,trọng tài nước ngoài là gì?"
})

# Giả sử raw_result là một list các dict, ta định dạng lại output:
formatted_output = ""
for item in raw_result:
    formatted_output += f"url: {item.get('url', '')}\n"
    formatted_output += f"content: {item.get('content', '')}\n"
    # Nếu kết quả có chứa link bổ sung, ví dụ 'link' thì in ra:
    if item.get('link'):
        formatted_output += f"link: {item.get('link')}\n"
    formatted_output += "-" * 50 + "\n"

print(formatted_output)


In [ ]:
# Sử dụng truy vấn tiếng Việt
raw_result = tool.invoke({
    "query": "Theo Luật, thông điệp dữ liệu được xem là có giá trị pháp lý như bản gốc khi đáp ứng những yêu cầu nào?"
})

# Giả sử raw_result là một list các dict, ta định dạng lại output:
formatted_output = ""
for item in raw_result:
    formatted_output += f"url: {item.get('url', '')}\n"
    formatted_output += f"content: {item.get('content', '')}\n"
    # Nếu kết quả có chứa link bổ sung, ví dụ 'link' thì in ra:
    if item.get('link'):
        formatted_output += f"link: {item.get('link')}\n"
    formatted_output += "-" * 50 + "\n"

print(formatted_output)

##